<a href="https://colab.research.google.com/github/Hrithik37/Classification/blob/master/cat_vs_dog_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2020-04-28 11:05:20--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 2404:6800:4003:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  58.3MB/s    in 1.4s    

2020-04-28 11:05:22 (58.3 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

last layer output shape:  (None, 7, 7, 768)


In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import regularizers
# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu',kernel_regularizer=regularizers.l2(0.001))(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.00001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])


In [ ]:
#!wget --no-check-certificate \
 #     -O /tmp/cats_and_dogs_filtered.zip

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
import zipfile

local_zip = '//tmp/cats_and_dogs_filtered.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')

zip_ref.extractall('/tmp')
zip_ref.close()

# Define our example directories and files
base_dir = '/tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')


train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1/255,
                                    rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 128,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 128,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))
#lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(0.001,
                                                            #decay_rate=1,
                                                            #staircase=False)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 100,
            epochs = 20,
            validation_steps = 50,
            verbose = 2)

Epoch 1/20
100/100 - 107s - loss: 2.1992 - accuracy: 0.9021 - val_loss: 1.9942 - val_accuracy: 0.9672
Epoch 2/20
100/100 - 106s - loss: 1.9943 - accuracy: 0.9421 - val_loss: 1.8805 - val_accuracy: 0.9703
Epoch 3/20
100/100 - 105s - loss: 1.8807 - accuracy: 0.9480 - val_loss: 1.7859 - val_accuracy: 0.9660
Epoch 4/20
100/100 - 104s - loss: 1.7733 - accuracy: 0.9578 - val_loss: 1.6946 - val_accuracy: 0.9709
Epoch 5/20
100/100 - 105s - loss: 1.6843 - accuracy: 0.9584 - val_loss: 1.6144 - val_accuracy: 0.9663
Epoch 6/20
100/100 - 105s - loss: 1.6037 - accuracy: 0.9617 - val_loss: 1.5442 - val_accuracy: 0.9653
Epoch 7/20
100/100 - 106s - loss: 1.5298 - accuracy: 0.9617 - val_loss: 1.4796 - val_accuracy: 0.9626
Epoch 8/20
100/100 - 106s - loss: 1.4621 - accuracy: 0.9649 - val_loss: 1.4222 - val_accuracy: 0.9639
Epoch 9/20
100/100 - 107s - loss: 1.4037 - accuracy: 0.9648 - val_loss: 1.3579 - val_accuracy: 0.9701
Epoch 10/20
100/100 - 107s - loss: 1.3470 - accuracy: 0.9676 - val_loss: 1.3064 - 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
epochs1 = range(len(loss))
plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

#plt.plot(epochs1,loss,'r',label = 'Training Loss')
#plot.plot(epochs1,loss,'b',label = 'validation loss' )
plt.legend(loc=0)
plt.figure()


plt.show()

NameError: ignored

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image


uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a cat")
  else:
    print(fn + " is a dog")

Saving cat.jpg to cat.jpg
[1.]
cat.jpg is a cat
